In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
import warnings

In [2]:
warnings.filterwarnings("ignore")
np.random.seed(42)


S0 = 100.0
r = 0.02
sigma = 0.20
T = 1.0

n_paths = 50_000
n_steps = 252

dt = T / n_steps
sqrt_dt = np.sqrt(dt)


In [3]:
def bs_call(S, K, r, vol, T):
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)



In [4]:
def simulate_cev(S0, r, sigma, gamma, T, n_paths, n_steps):
    paths = np.zeros((n_paths, n_steps + 1))
    paths[:, 0] = S0

    Z = np.random.randn(n_paths, n_steps)

    for t in range(n_steps):
        S = paths[:, t]
        drift = r * S * dt
        diffusion = sigma * np.power(np.maximum(S, 1e-8), gamma) * sqrt_dt * Z[:, t]
        paths[:, t + 1] = np.maximum(S + drift + diffusion, 1e-8)

    return paths


In [5]:
def price_call_mc(paths, K, r, T):
    ST = paths[:, -1]
    payoff = np.maximum(ST - K, 0.0)
    discounted = np.exp(-r * T) * payoff

    price = discounted.mean()
    stderr = discounted.std(ddof=1) / np.sqrt(len(discounted))
    ci = 1.96 * stderr

    return {
        "price": price,
        "bs_price": bs_call(S0, K, r, sigma, T) ,
        "stderr": stderr,
        "ci_low": price - ci,
        "ci_high": price + ci
    }


In [6]:
def get_implied_vol(target_price, S, K, r, T):
    def f(vol):
        return bs_call(S, K, r, vol, T) - target_price

    try:
        return brentq(f, 0.01, 2.0)
    except ValueError:
        return np.nan



In [7]:
gammas = [0.5, 1.0, 1.5]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, g in zip(axes, gammas):
    paths = simulate_cev(S0, r, sigma, g, T, 50, n_steps)
    for p in paths:
        ax.plot(p, alpha=0.7)
    ax.set_title(f"γ = {g}")
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Stock Price")
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

2026-01-07 22:45:17.691 Python[3571:158102] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/10/n59cq8_939x8yv9swss8dkp00000gn/T/org.python.python.savedState


In [8]:
results = []
K = 100

for g in gammas:
    paths = simulate_cev(S0, r, sigma, g, T, n_paths, n_steps)
    res = price_call_mc(paths, K, r, T)
    res["gamma"] = g
    results.append(res)

df = pd.DataFrame(results)
print(df)

bs_price = bs_call(S0, K, r, sigma, T)
cev_price = df.loc[df.gamma == 1.0, "price"].values[0]

print("\nBlack–Scholes price:", round(bs_price, 4))
print("CEV price (γ=1):    ", round(cev_price, 4))


      price  bs_price    stderr    ci_low    ci_high  gamma
0  2.139993  8.916037  0.007685  2.124930   2.155055    0.5
1  8.907968  8.916037  0.061434  8.787557   9.028379    1.0
2  8.327138  8.916037  2.011518  4.384564  12.269713    1.5

Black–Scholes price: 8.916
CEV price (γ=1):     8.908


In [9]:
gamma_smile = 1.5
strikes = [80, 90, 100, 110, 120]

paths = simulate_cev(S0, r, sigma, gamma_smile, T, n_paths, n_steps)

ivs = []

In [10]:

for K_i in strikes:
    paths = simulate_cev(S0, r, sigma, gamma_smile, T, 200_000, n_steps)
    price = price_call_mc(paths, K_i, r, T)["price"]
    intrinsic = max(S0 - K_i * np.exp(-r * T), 0)

    print(f"K={K_i}, price={price:.4f}, intrinsic={intrinsic:.4f}")

    ivs.append(get_implied_vol(price, S0, K_i, r, T))


K=80, price=6.5308, intrinsic=21.5841
K=90, price=5.8646, intrinsic=11.7821
K=100, price=8.5790, intrinsic=1.9801
K=110, price=13.7057, intrinsic=0.0000
K=120, price=4.2147, intrinsic=0.0000


In [11]:
plt.figure(figsize=(7, 5))
plt.plot(strikes, np.array(ivs) * 100, marker="o")
plt.axhline(sigma * 100, linestyle="--", color="gray")
plt.xlabel("Strike")
plt.ylabel("Implied Volatility (%)")
plt.title("Implied Volatility Skew (γ = 0.5)")
plt.grid(alpha=0.3)
plt.show()